In [82]:
import pandas
import numpy

data = pandas.read_json("db.json")
data = data.T
print(data.head())
print(data.shape)

    app_id                                               tags  \
1   536280  [FPS, PvE, Vehicular Combat, RTS, Shooter, Str...   
2   982800  [Strategy, Simulation, Indie, Management, Sing...   
3   767960  [Strategy, Indie, Education, RTS, Real-Time, S...   
4  1368670  [Sports, Action, Arcade, Character Action Game...   
5   977800  [Strategy, Adventure, Free to Play, RPG, Indie...   

                                     reviews  
1  {'overall': {'rating': 61, 'count': 120}}  
2           {'overall': 'need_more_reviews'}  
3           {'overall': 'need_more_reviews'}  
4           {'overall': 'need_more_reviews'}  
5           {'overall': 'need_more_reviews'}  
(76227, 3)


In [83]:
data.loc[(data['reviews'] == {'overall': 'need_more_reviews'}),'reviews'] = numpy.nan
data = pandas.concat([data.drop(['reviews'], axis=1), data['reviews'].apply(pandas.Series)], axis=1)
del data[0]
del data["recent"]
print(data.head())

    app_id                                               tags  \
1   536280  [FPS, PvE, Vehicular Combat, RTS, Shooter, Str...   
2   982800  [Strategy, Simulation, Indie, Management, Sing...   
3   767960  [Strategy, Indie, Education, RTS, Real-Time, S...   
4  1368670  [Sports, Action, Arcade, Character Action Game...   
5   977800  [Strategy, Adventure, Free to Play, RPG, Indie...   

                        overall  
1  {'rating': 61, 'count': 120}  
2                           NaN  
3                           NaN  
4                           NaN  
5                           NaN  


In [84]:
data = pandas.concat([data.drop(['overall'], axis=1), data['overall'].apply(pandas.Series)], axis=1)
del data[0]
data.rename(columns = {'count':'review_count'}, inplace = True)
print(data.head())

    app_id                                               tags  review_count  \
1   536280  [FPS, PvE, Vehicular Combat, RTS, Shooter, Str...         120.0   
2   982800  [Strategy, Simulation, Indie, Management, Sing...           NaN   
3   767960  [Strategy, Indie, Education, RTS, Real-Time, S...           NaN   
4  1368670  [Sports, Action, Arcade, Character Action Game...           NaN   
5   977800  [Strategy, Adventure, Free to Play, RPG, Indie...           NaN   

   rating  
1    61.0  
2     NaN  
3     NaN  
4     NaN  
5     NaN  


In [85]:
# Splitting the dataset into two to make operations for tags easier:
game_info = data.copy()
del game_info['tags']
tags = pandas.concat([data.drop(['tags'], axis=1), data['tags'].apply(pandas.Series)], axis=1)
del tags['app_id']
del tags['review_count']
del tags['rating']
# Creating a column for each tag:
tags = pandas.get_dummies(tags)
tags.rename(columns=lambda x: x.split("_")[1], inplace=True)
tags = tags.groupby(level=0, axis=1).sum()
print(game_info.head())
print(tags.head())

    app_id  review_count  rating
1   536280         120.0    61.0
2   982800           NaN     NaN
3   767960           NaN     NaN
4  1368670           NaN     NaN
5   977800           NaN     NaN
   1980s  1990's  2.5D  2D  2D Fighter  2D Platformer  360 Video  3D  \
1      0       0     0   0           0              0          0   0   
2      0       0     0   0           0              0          0   0   
3      0       0     0   0           0              0          0   0   
4      0       0     1   1           0              0          0   0   
5      0       0     0   0           0              0          0   0   

   3D Fighter  3D Platformer  ...  Web Publishing  Well-Written  Werewolves  \
1           0              0  ...               0             0           0   
2           0              0  ...               0             0           0   
3           0              0  ...               0             0           0   
4           0              0  ...               0    

In [86]:
# Merging the tables back together:
games = game_info.merge(tags, left_index=True, right_index=True)
print(games.head())
print(games.shape)

    app_id  review_count  rating  1980s  1990's  2.5D  2D  2D Fighter  \
1   536280         120.0    61.0      0       0     0   0           0   
2   982800           NaN     NaN      0       0     0   0           0   
3   767960           NaN     NaN      0       0     0   0           0   
4  1368670           NaN     NaN      0       0     1   1           0   
5   977800           NaN     NaN      0       0     0   0           0   

   2D Platformer  360 Video  ...  Web Publishing  Well-Written  Werewolves  \
1              0          0  ...               0             0           0   
2              0          0  ...               0             0           0   
3              0          0  ...               0             0           0   
4              0          0  ...               0             0           0   
5              0          0  ...               0             0           0   

   Western  Word Game  World War I  World War II  Wrestling  Zombies  eSports  
1        0  

In [87]:
# Checking if the tags seem right for tag "Strategy":
print(games["Strategy"].head())
# They seem to be right.

1    1
2    1
3    1
4    0
5    1
Name: Strategy, dtype: uint8


In [88]:
# Checking for nas:
print(games.isnull().sum())
# For more than half of the games review data is missing.

app_id              0
review_count    43746
rating          43746
1980s               0
1990's              0
                ...  
World War I         0
World War II        0
Wrestling           0
Zombies             0
eSports             0
Length: 425, dtype: int64
